###  SIMULACIÓN JUEGO GUAYABITA 
##### Universidad Externado
##### Seminario de Programación
##### Liliana Hernández - Anjely Amazo - Julieth Perea

Un juego tradicional de los niños que se la pasaban en las calles es el famoso Guayabita, consiste en una apuesta que se hace con un dado y podría ser muy adictivo incluso para los grandes, planteamos una simulación de el juego en un casino online

In [1]:
import numpy as np
import pandas as pd
from random import randint, choice

In [2]:
pote = 0
casino = 0
bolsillo= 0
tamanio= 0
estado=None
T_Ronda = 0

In [3]:
def lanzamiento():
    return randint(1,6)

In [4]:
def apuesta_riesgo(lanzamiento):
    riesgo = None
    if (lanzamiento == 2):
        riesgo = 'MAYOR'
    elif (lanzamiento == 5):
        riesgo = 'MENOR'
    else:
        riesgo_aleatorio = ['MAYOR', 'MENOR']
        riesgo = choice(riesgo_aleatorio)
    return riesgo

In [5]:
def juega():
    jugar = [True, False]
    return choice(jugar)

In [6]:
def monto_apuesta(valor_apuesta):
    global pote
    pote = pote+valor_apuesta

In [7]:
def retracto_apuesta(valor_apuesta):
    global pote
    pote =pote-valor_apuesta

In [8]:
def jugador(valor_apuesta):
    global pote,bolsillo,casino,estado
    jugar = juega()  #Cada jugador empieza su turno indicando si apuesta, si es así paga el valor acordado
    Desicion_Apuesta=jugar
    primer_lanzamiento=None
    segundo_lanzamiento= None
    riesgo= None
    casino = 0
    bolsillo = 0
    estado='No juega'
    if (jugar):
        monto_apuesta(valor_apuesta)
        primer_lanzamiento = lanzamiento() # El jugador lanza el dado, luego, según su lanzamiento el puede apostar 
        # a obtener un número MAYOR o MENOR   
        if (primer_lanzamiento == 1): # Sí el primer resultado es 1 el jugador pierde la apuesta y pierde el turno
            estado='Pierde turno'
            bolsillo = 0
        elif (primer_lanzamiento == 6):# Sí el primer resultado es 6 el jugador puede retractarse de su apuesta y pasar el turno
            bolsillo = 0
            if (not juega()):
                pote=pote-valor_apuesta
                Desicion_Apuesta='Retracto'
                estado='Retracto'
            else:
                estado='Pierde turno'
        else:
            riesgo = apuesta_riesgo(primer_lanzamiento)
            segundo_lanzamiento = lanzamiento()
            if (
                ((riesgo == 'MAYOR') & (primer_lanzamiento < segundo_lanzamiento)) 
                or ((riesgo == 'MENOR') & (primer_lanzamiento > segundo_lanzamiento))
            ):
                bolsillo = pote*0.95 
                casino = pote*0.05  # Cada vez que se reclame el pote , el casino gana el 5%
            else:
                estado='Pierde'
    return Desicion_Apuesta,primer_lanzamiento,segundo_lanzamiento,riesgo,pote,bolsillo,casino

In [9]:
def ronda(Cant_Jugadores,valor_apuesta,InicioMesa, Mesa,numRonda):
    global DF_CASINO,pote,bolsillo,casino,estado,T_Ronda
    i = 0
    media_minutos= int(20/Cant_Jugadores)  #Como el ennunciado dice que una ronda dura aproximadamente 20 minutos se calcula
    #la media en función de la cantidad de jugadores que participan en la ronda
    Exp_lista=np.random.exponential(media_minutos,Cant_Jugadores)  ###  Arreglo de tiempos en minutos de cada juego, 
    # con valores aleatorios con distribución exponencial
    T_Jugador= 0
    bolsillo = 0
    while i < Cant_Jugadores:
        T_Jugador=int(Exp_lista[i]) + 1 ##Sumo UN minuto para asegurar que el valor mínimo aleatoreo sea de 1min
        T_Ronda=T_Ronda + T_Jugador
        Desicion_Apuesta,primer_lanzamiento,segundo_lanzamiento,riesgo,pote2,bolsillo2,casino2 = jugador(valor_apuesta)
        i=i+1
        if T_Ronda > 20:
            casino2=pote
            pote2=0
            bolsillo2 =0
            estado='Tiempo Excedido'
            DF_CASINO = DF_CASINO.append({'Mesa': Mesa, 'HoraInicio':InicioMesa,'Ronda':numRonda,'Jugador':i,'Estado':estado,                                          
                              'Apuesta':Desicion_Apuesta,'D1':primer_lanzamiento,'D2':segundo_lanzamiento,
                              'Eleccion':riesgo,'Tiempo':T_Jugador,'T_Acum':T_Ronda,
                              'VlrPote':pote2,'VlrJugador':bolsillo2,'VlrCasino':casino2}, ignore_index=True)
            break  ## Finaliza cuando se agote el tiempo aproximado de 20 minutos, en este caso, el casino se queda con el pote
        elif (bolsillo > 0) :
            estado='Ganador'
            DF_CASINO = DF_CASINO.append({'Mesa': Mesa, 'HoraInicio':InicioMesa,'Ronda':numRonda,'Jugador':i,'Estado':estado,
                              'Apuesta':Desicion_Apuesta,'D1':primer_lanzamiento,'D2':segundo_lanzamiento,
                              'Eleccion':riesgo,'Tiempo':T_Jugador,'T_Acum':T_Ronda,
                              'VlrPote':pote2,'VlrJugador':bolsillo2,'VlrCasino':casino2}, ignore_index=True)
            break  ## Finaliza la ronda cuando alguien reclame el pote
        DF_CASINO = DF_CASINO.append({'Mesa': Mesa, 'HoraInicio':InicioMesa,'Ronda':numRonda,'Jugador':i,'Estado':estado,
                              'Apuesta':Desicion_Apuesta,'D1':primer_lanzamiento,'D2':segundo_lanzamiento,
                              'Eleccion':riesgo,'Tiempo':T_Jugador,'T_Acum':T_Ronda,
                              'VlrPote':pote2,'VlrJugador':bolsillo2,'VlrCasino':casino2}, ignore_index=True)

In [10]:
def mesa(InicioMesa, Mesa, valor_apuesta,tamanio):
    global pote, bolsillo,T_Ronda,estado
    bolsillo = 0
    pote= tamanio*valor_apuesta # Para iniciar el juego cada jugador debe pagar su ingreso, con el valor apostado
    w=0
    estado = 'Inicio Mesa'
    while T_Ronda <= 20:
        w=w+1
        if estado in ['Ganador','Tiempo Excedido']: break
        ronda(tamanio,valor_apuesta,InicioMesa, Mesa,w)

In [29]:
Poisson_list=np.random.poisson(72,24) # se estima una media de ingreso de 72 segundos (50 usuarios en 3600 segundos)
h=0
Lista_llegadas=[]
for i in Poisson_list: # Se simula la llegada de los jugadores con una distribución de Poisson
    if h < 3 or h> 15: #Se simula un casino que funciona durante 11 horas, de las 16:00 hasta las 03:00
        Min=np.random.choice(np.arange(0,60),i)
        Min.sort()
        Lista_llegadas.extend([(h,j,(h-1)*60+j) for j in Min])
    h+=1

Lista_tiempos=[]
DF=pd.DataFrame(Lista_llegadas)
DF.columns=['Hora','Minuto','Minutos desde apertura']

tam=tamanio-1

### Simulación 1
Se fija un valor para apostar durante toda la ronda, una mesa con 4 jugadores que reconocen la probabilidad de ganar después del primer lanzamiento y que la probabilidad de su apuesta depende que el valor supere 0.5.

In [30]:
valor_apuesta = 10000
tamanio=4

In [31]:
DF_CASINO=pd.DataFrame(columns=['Mesa','HoraInicio','Ronda','Jugador','Estado','Apuesta','D1','D2','Eleccion','Tiempo','T_Acum','VlrPote','VlrJugador','VlrCasino'])
q=1
for i in DF.index:
    if i%tamanio==tam:
        HoraInicio=str(DF['Hora'][i])+':'+str(DF['Minuto'][i])
        T_Ronda=0
        mesa(HoraInicio, q, valor_apuesta,tamanio)
        q+=1
        
DF_CASINO

,Mesa,HoraInicio,Ronda,Jugador,Estado,Apuesta,D1,D2,Eleccion,Tiempo,T_Acum,VlrPote,VlrJugador,VlrCasino
0,1,0:4,1,1,Ganador,True,2,3,MAYOR,2,2,50000,47500.0,2500.0
1,2,0:11,1,1,No juega,False,None,None,None,1,1,40000,0.0,0.0
2,2,0:11,1,2,Pierde turno,True,1,None,None,2,3,50000,0.0,0.0
3,2,0:11,1,3,Pierde turno,True,6,None,None,7,10,60000,0.0,0.0
4,2,0:11,1,4,Pierde,True,5,1,MAYOR,1,11,70000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,189,23:57,1,1,No juega,False,None,None,None,1,1,40000,0.0,0.0
610,189,23:57,1,2,No juega,False,None,None,None,4,5,40000,0.0,0.0
611,189,23:57,1,3,No juega,False,None,None,None,6,11,40000,0.0,0.0
612,189,23:57,1,4,Retracto,Retracto,6,None,None,1,12,40000,0.0,0.0


### Simulación 2
Una mesa con 4 jugadores menos controlados que reconocen la probabilidad de ganar después del primer lanzamiento y que la probabilidad de su apuesta depende de un valor aleatoreo que se escoge según una distribución normal con media 0.6 y desviación estandar 0.1.

In [32]:
def apuesta_riesgo(lanzamiento):
    riesgo = None
    if (lanzamiento == 2) or (lanzamiento == 3) :
        riesgo_aleatorio = ['MAYOR','MAYOR','MAYOR','MAYOR','MAYOR','MAYOR', 'MENOR', 'MENOR', 'MENOR', 'MENOR']
        riesgo = choice(riesgo_aleatorio)
    elif (lanzamiento == 4) or (lanzamiento == 5):
        riesgo_aleatorio = ['MAYOR','MAYOR','MAYOR','MAYOR' 'MENOR','MENOR','MENOR','MENOR','MENOR','MENOR']
        riesgo = choice(riesgo_aleatorio)
    return riesgo

In [33]:
Poisson_list=np.random.poisson(72,24) # se estima una media de ingreso de 72 segundos (50 usuarios en 3600 segundos)
h=0
Lista_llegadas=[]
for i in Poisson_list:    
    if h < 3 or h> 15:
        Min=np.random.choice(np.arange(0,60),i)
        Min.sort()
        Lista_llegadas.extend([(h,j,(h-1)*60+j) for j in Min])
    h+=1

Lista_tiempos=[]
DF=pd.DataFrame(Lista_llegadas)
DF.columns=['Hora','Minuto','Minutos desde apertura']

tam=tamanio-1

valor_apuesta = 10000
tamanio=4

In [34]:
DF_CASINO=pd.DataFrame(columns=['Mesa','HoraInicio','Ronda','Jugador','Estado','Apuesta','D1','D2','Eleccion','Tiempo','T_Acum','VlrPote','VlrJugador','VlrCasino'])
q=1
for i in DF.index:
    if i%tamanio==tam:
        HoraInicio=str(DF['Hora'][i])+':'+str(DF['Minuto'][i])
        T_Ronda=0
        mesa(HoraInicio, q, valor_apuesta,tamanio)
        q+=1
        
DF_CASINO

,Mesa,HoraInicio,Ronda,Jugador,Estado,Apuesta,D1,D2,Eleccion,Tiempo,T_Acum,VlrPote,VlrJugador,VlrCasino
0,1,0:1,1,1,Pierde,True,3,3,MENOR,9,9,50000,0,0
1,1,0:1,1,2,Pierde,True,4,1,MAYORMENOR,8,17,60000,0,0
2,1,0:1,1,3,No juega,False,None,None,None,1,18,60000,0,0
3,1,0:1,1,4,Ganador,True,3,6,MAYOR,2,20,70000,66500,3500
4,2,0:5,1,1,No juega,False,None,None,None,4,4,40000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,203,23:56,1,4,No juega,False,None,None,None,2,16,60000,0,0
708,203,23:56,2,1,Tiempo Excedido,False,None,None,None,7,23,0,0,60000
709,204,23:59,1,1,Pierde turno,True,1,None,None,9,9,50000,0,0
710,204,23:59,1,2,No juega,False,None,None,None,1,10,50000,0,0


### Simulación 3
Se fija un valor para apostar durante toda la ronda, una mesa con 6 jugadores que reconocen la probabilidad de ganar después del primer lanzamiento y que la probabilidad de su apuesta depende que el valor supere 0.5.

In [35]:
def apuesta_riesgo(lanzamiento):
    riesgo = None
    if (lanzamiento == 2):
        riesgo = 'MAYOR'
    elif (lanzamiento == 5):
        riesgo = 'MENOR'
    else:
        riesgo_aleatorio = ['MAYOR', 'MENOR']
        riesgo = choice(riesgo_aleatorio)
    return riesgo

In [36]:
Poisson_list=np.random.poisson(72,24) # se estima una media de ingreso de 72 segundos (50 usuarios en 3600 segundos)
h=0
Lista_llegadas=[]
for i in Poisson_list:    
    if h < 3 or h> 15:
        Min=np.random.choice(np.arange(0,60),i)
        Min.sort()
        Lista_llegadas.extend([(h,j,(h-1)*60+j) for j in Min])
    h+=1

Lista_tiempos=[]
DF=pd.DataFrame(Lista_llegadas)
DF.columns=['Hora','Minuto','Minutos desde apertura']

tam=tamanio-1

valor_apuesta = 10000
tamanio=6

In [40]:
DF_CASINO=pd.DataFrame(columns=['Mesa','HoraInicio','Ronda','Jugador','Estado','Apuesta','D1','D2','Eleccion','Tiempo','T_Acum','VlrPote','VlrJugador','VlrCasino'])
q=1
for i in DF.index:
    if i%tamanio==tam:
        HoraInicio=str(DF['Hora'][i])+':'+str(DF['Minuto'][i])
        T_Ronda=0
        mesa(HoraInicio, q, valor_apuesta,tamanio)
        q+=1
        
DF_CASINO

,Mesa,HoraInicio,Ronda,Jugador,Estado,Apuesta,D1,D2,Eleccion,Tiempo,T_Acum,VlrPote,VlrJugador,VlrCasino
0,1,0:5,1,1,No juega,False,None,None,None,2,2,60000,0,0
1,1,0:5,1,2,No juega,False,None,None,None,17,19,60000,0,0
2,1,0:5,1,3,Tiempo Excedido,False,None,None,None,20,39,0,0,60000
3,2,0:11,1,1,No juega,False,None,None,None,3,3,60000,0,0
4,2,0:11,1,2,No juega,False,None,None,None,3,6,60000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,132,23:59,1,1,No juega,False,None,None,None,1,1,60000,0,0
542,132,23:59,1,2,Pierde,True,4,4,MENOR,1,2,70000,0,0
543,132,23:59,1,3,Pierde,True,2,2,MAYOR,9,11,80000,0,0
544,132,23:59,1,4,No juega,False,None,None,None,2,13,80000,0,0


### Simulación 4
Una mesa con 6 jugadores menos controlados que reconocen la probabilidad de ganar después del primer lanzamiento y que la probabilidad de su apuesta depende de un valor aleatoreo que se escoge según una distribución normal con media 0.6 y desviación estandar 0.1.

In [41]:
def apuesta_riesgo(lanzamiento):
    riesgo = None
    if (lanzamiento == 2) or (lanzamiento == 3) :
        riesgo_aleatorio = ['MAYOR','MAYOR','MAYOR','MAYOR','MAYOR','MAYOR', 'MENOR', 'MENOR', 'MENOR', 'MENOR']
        riesgo = choice(riesgo_aleatorio)
    elif (lanzamiento == 4) or (lanzamiento == 5):
        riesgo_aleatorio = ['MAYOR','MAYOR','MAYOR','MAYOR' 'MENOR','MENOR','MENOR','MENOR','MENOR','MENOR']
        riesgo = choice(riesgo_aleatorio)
    return riesgo

In [42]:
Poisson_list=np.random.poisson(72,24) # se estima una media de ingreso de 72 segundos (50 usuarios en 3600 segundos)
h=0
Lista_llegadas=[]
for i in Poisson_list:    
    if h < 3 or h> 15:
        Min=np.random.choice(np.arange(0,60),i)
        Min.sort()
        Lista_llegadas.extend([(h,j,(h-1)*60+j) for j in Min])
    h+=1

Lista_tiempos=[]
DF=pd.DataFrame(Lista_llegadas)
DF.columns=['Hora','Minuto','Minutos desde apertura']

tam=tamanio-1

valor_apuesta = 10000
tamanio=6

In [43]:
DF_CASINO=pd.DataFrame(columns=['Mesa','HoraInicio','Ronda','Jugador','Estado','Apuesta','D1','D2','Eleccion','Tiempo','T_Acum','VlrPote','VlrJugador','VlrCasino'])
q=1
for i in DF.index:
    if i%tamanio==tam:
        HoraInicio=str(DF['Hora'][i])+':'+str(DF['Minuto'][i])
        T_Ronda=0
        mesa(HoraInicio, q, valor_apuesta,tamanio)
        q+=1
        
DF_CASINO

,Mesa,HoraInicio,Ronda,Jugador,Estado,Apuesta,D1,D2,Eleccion,Tiempo,T_Acum,VlrPote,VlrJugador,VlrCasino
0,1,0:5,1,1,No juega,False,None,None,None,2,2,60000,0,0
1,1,0:5,1,2,No juega,False,None,None,None,3,5,60000,0,0
2,1,0:5,1,3,Ganador,True,4,5,MAYOR,2,7,70000,66500,3500
3,2,0:11,1,1,Ganador,True,2,3,MAYOR,5,5,70000,66500,3500
4,3,0:18,1,1,Pierde turno,True,1,None,None,1,1,70000,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,125,23:52,1,6,No juega,False,None,None,None,3,17,80000,0,0
484,125,23:52,2,1,Tiempo Excedido,False,None,None,None,7,24,0,0,80000
485,126,23:56,1,1,Pierde,True,3,3,MAYOR,7,7,70000,0,0
486,126,23:56,1,2,No juega,False,None,None,None,3,10,70000,0,0


### Simulación 5
Se puede escoger el valor de la apuesta y el tamaño de la mesa

In [47]:
def apuesta_riesgo(lanzamiento):
    riesgo = None
    if (lanzamiento == 2):
        riesgo = 'MAYOR'
    elif (lanzamiento == 5):
        riesgo = 'MENOR'
    else:
        riesgo_aleatorio = ['MAYOR', 'MENOR']
        riesgo = choice(riesgo_aleatorio)
    return riesgo

In [48]:
Poisson_list=np.random.poisson(72,24) # se estima una media de ingreso de 72 segundos (50 usuarios en 3600 segundos)
h=0
Lista_llegadas=[]
for i in Poisson_list:    
    if h < 3 or h> 15:
        Min=np.random.choice(np.arange(0,60),i)
        Min.sort()
        Lista_llegadas.extend([(h,j,(h-1)*60+j) for j in Min])
    h+=1

Lista_tiempos=[]
DF=pd.DataFrame(Lista_llegadas)
DF.columns=['Hora','Minuto','Minutos desde apertura']

tam=tamanio-1

valor_apuesta = int(input('Ingrese el valor de la apuesta'))
tamanio=int(input('Ingrese el tamaño de la mesa'))

Ingrese el valor de la apuesta5000
Ingrese el tamaño de la mesa6


In [49]:
DF_CASINO=pd.DataFrame(columns=['Mesa','HoraInicio','Ronda','Jugador','Estado','Apuesta','D1','D2','Eleccion','Tiempo','T_Acum','VlrPote','VlrJugador','VlrCasino'])
q=1
for i in DF.index:
    if i%tamanio==tam:
        HoraInicio=str(DF['Hora'][i])+':'+str(DF['Minuto'][i])
        T_Ronda=0
        mesa(HoraInicio, q, valor_apuesta,tamanio)
        q+=1
        
DF_CASINO

,Mesa,HoraInicio,Ronda,Jugador,Estado,Apuesta,D1,D2,Eleccion,Tiempo,T_Acum,VlrPote,VlrJugador,VlrCasino
0,1,0:3,1,1,No juega,False,None,None,None,1,1,30000,0,0
1,1,0:3,1,2,No juega,False,None,None,None,1,2,30000,0,0
2,1,0:3,1,3,Retracto,Retracto,6,None,None,4,6,30000,0,0
3,1,0:3,1,4,Pierde,True,2,1,MAYOR,1,7,35000,0,0
4,1,0:3,1,5,Ganador,True,5,3,MENOR,3,10,40000,38000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,134,23:50,1,3,No juega,False,None,None,None,2,8,35000,0,0
587,134,23:50,1,4,No juega,False,None,None,None,4,12,35000,0,0
588,134,23:50,1,5,No juega,False,None,None,None,3,15,35000,0,0
589,134,23:50,1,6,Ganador,True,2,5,MAYOR,1,16,40000,38000,2000


#### Las ganacias del casino se almacenan en la columna CASINO del dataframe para cada simulación